### Using Rbdyn and SpaceVecAlg
1. The library comes with python bindings using cython transcription. Thus, fast prototyping can be done to verify algorithms using the provide python functions.

2. To setup the libraries, we will need to install some common components first.
<br>`sudo apt install apt-transport-https lsb-release`

3. *Eigen3ToPython*
<br>1) `sudo apt-key adv --keyserver 'hkp://keyserver.ubuntu.com:80' --recv-key 892EA6EE273707C6495A6FB6220D644C64666806`
<br>2) `sudo sh -c 'echo "deb https://dl.bintray.com/gergondet/multi-contact-release $(lsb_release -sc) main" | sudo tee -a /etc/apt/sources.list.d/multi-contact.list'`
<br>3) `sudo apt update && sudo apt install python-eigen python3-eigen`

4. *SpaceVecAlg*
<br>1) `sudo apt update && sudo apt install libspacevecalg-dev python-spacevecalg python3-spacevecalg`

5. *RBdyn*
<br>1) `sudo apt update && sudo apt install librbdyn-dev python-rbdyn python3-rbdyn`

In [ ]:
import pybullet as b
import time
import pybullet_data
import eigen as e
import rbdyn as rbd
import sva as s
import numpy as np
from rbdyn.parsers import *

DT = 0.001 #s
MOVEMENT_TIME = 10000 #ms
URDF = "./resource/kuka_iiwa.urdf"

def body_id_from_name(name, bodies):
    '''Gets the body Id from the body name
    @param name The name of the body
    @param bodies The set of bodies provided by the multibody data structure
    @return Id of the body, -1 if not found
    '''
    for bi, b in enumerate(bodies):
        if (b.name().decode("utf-8") == name):
            return bi
    return -1

def sva_to_affine(s):
    '''Converts a spatial transform matrix to a homogeneous transform matrix
    @param s Spatial transform
    @return Homogeneous transform matrix
    '''
    m4d = e.Matrix4d.Identity()
    R = s.rotation().transpose()
    p = s.translation();
    
    for row in range(3):
        for col in range(3):
            m4d.coeff(row, col, R.coeff(row, col))
    for row in range(3):
        m4d.coeff(row, 3, p[row]);
        
    return m4d

def inv_spd_sr(mat, k=1e-4):
    '''Converts an inverse into an SVD problem with singularity robust inverse
    @param mat Matrix to be inversed
    @param tolerance Tolerance to the minimum singular value that can go through reciprocal operation
    @return Homogeneous transform matrix
    '''
    mat_np = np.array(mat)
    reg = k * np.identity(np.size(mat_np,0))
    
    return e.MatrixXd(mat_np.transpose().dot(np.linalg.inv(mat_np.dot(mat_np.transpose()) + reg)))

def inv_spd(mat, tolerance=1e-4):
    '''Converts an inverse into an SVD problem with singularity handling
    @param mat Matrix to be inversed
    @param tolerance Tolerance to the minimum singular value that can go through reciprocal operation
    @return Homogeneous transform matrix
    '''
    mat_np = np.array(mat)
    u, s, vt = np.linalg.svd(mat_np, full_matrices=True, hermitian=True)

    sinv = np.zeros(s.shape)
    half_tolerance = tolerance/2;
     
    for i in range(len(s)):
        if s[i] > tolerance:
            sinv[i] = 1/s[i]
        elif s[i] > half_tolerance:
            sinv[i] = (s[i]-half_tolerance)/half_tolerance * (1/tolerance)
        else:
            sinv[i] = 0

    return e.MatrixXd(vt.transpose()@np.diag(sinv)@u.transpose())

def get_q(self):
    """
    Returns the joint angle q
    @param urdf Path for urdf file
    """
    q = np.zeros(self.dyn.mb.nrDof())
    for i in range(0, self.dyn.mb.nrDof()):
        q[i] = self.dyn.mbc.q[i+1][0]
    return q

def set_q(self, q):
    """
    Sets the joint angles q
    @param q Joint angles
    """
    for i in range(0, self.dyn.mb.nrDof()):
        self.dyn.mbc.q[i+1][0] = q[i]
    return q
    
# import robot as global variable
p = from_urdf_file(URDF)
print("Imported " + p.name.decode("utf-8"))
# set gravity direction (this is the acceleration at base joint for RNEA)
p.mbc.gravity = e.Vector3d(0,0,9.81)
p.mbc.zero(p.mb)

# setup physics
physicsClient = b.connect(b.GUI)
b.setAdditionalSearchPath(pybullet_data.getDataPath()) #used by loadURDF
b.setGravity(0,0,-9.81)
b.setRealTimeSimulation(0)
b.setTimeStep(DT)

# import robot
planeId = b.loadURDF("plane.urdf")
startPos = [0,0,0]
startOrientation = b.getQuaternionFromEuler([0,0,0])
loadFlag = b.URDF_USE_INERTIA_FROM_FILE | b.URDF_USE_SELF_COLLISION_EXCLUDE_ALL_PARENTS
robotId = b.loadURDF(URDF,startPos, startOrientation, flags=loadFlag)
nDof = b.getNumJoints(robotId)
jointsId = range(nDof)

# selection matrix to swap linear and angular components
S = e.MatrixXd(6,6)
S.coeff(0,3,1)
S.coeff(1,4,1)
S.coeff(2,5,1)
S.coeff(3,0,1)
S.coeff(4,1,1)
S.coeff(5,2,1)

# task definitions
bodyName1 = "lbr_iiwa_link_7"
S1 = e.MatrixXd(3,6)
S1.coeff(0,3,1)
S1.coeff(1,4,1)
S1.coeff(2,5,1)

bodyName2 = "lbr_iiwa_link_3"
S2 = e.MatrixXd(3,6)
S2.coeff(0,3,1)
S2.coeff(1,4,1)
S2.coeff(2,5,1)

# setup configuration
b.setJointMotorControlArray(robotId, jointsId, b.VELOCITY_CONTROL, forces=[0] * nDof)
q_home = 0.1*np.arange(nDof)
for i in jointsId:
    b.resetJointState(robotId, i, targetValue=q_home[i])
q_home = e.VectorXd(q_home)
    
# simulate torque control
for i in range(10000):
    joint_states = b.getJointStates(robotId, jointsId)
    
    # update q and dq states from simulation
    q = [joint_states[i][0] for i in jointsId]
    dq = [joint_states[i][1] for i in jointsId]
    for i in range(0, nDof):
        p.mbc.q[i+1][0] = q[i]
        p.mbc.alpha[i+1][0] = dq[i]

    # forward kinematics
    rbd.forwardKinematics(p.mb, p.mbc)
    rbd.forwardVelocity(p.mb, p.mbc)
    
    # convert
    q = e.VectorXd(q)
    dq = e.VectorXd(dq)
    
    # pose
    ofsRot = e.Matrix3d.Identity()
    ofsPos = e.Vector3d(1,0,0)
    T1 = sva_to_affine(s.PTransformd(ofsRot.transpose(), ofsPos) * p.mbc.bodyPosW[body_id_from_name(bodyName1, p.mb.bodies())])
    T2 = sva_to_affine(s.PTransformd(ofsRot.transpose(), ofsPos) * p.mbc.bodyPosW[body_id_from_name(bodyName2, p.mb.bodies())])
    
    # jacobian, dJacobian [angular; linear]
    jac = rbd.Jacobian(p.mb, bodyName1.encode('utf-8'), ofsPos)
    tempJ1 = e.MatrixXd(6, p.mb.nrDof());
    tempdJ1 = e.MatrixXd(6, p.mb.nrDof());
    jac.fullJacobian(p.mb, jac.jacobian(p.mb, p.mbc), tempJ1);
    jac.fullJacobian(p.mb, jac.jacobianDot(p.mb, p.mbc), tempdJ1);
    J1 = S1*tempJ1
    dJ1 = S1*tempdJ1
    
    # jacobian, dJacobian [angular; linear]
    jac = rbd.Jacobian(p.mb, bodyName2.encode('utf-8'), ofsPos)
    tempJ2 = e.MatrixXd(6, p.mb.nrDof());
    tempdJ2 = e.MatrixXd(6, p.mb.nrDof());
    jac.fullJacobian(p.mb, jac.jacobian(p.mb, p.mbc), tempJ2);
    jac.fullJacobian(p.mb, jac.jacobianDot(p.mb, p.mbc), tempdJ2);
    J2 = S2*tempJ2
    dJ2 = S2*tempdJ2
    
    # mass matrix
    fd = rbd.ForwardDynamics(p.mb)
    fd.computeH(p.mb, p.mbc);
    M = fd.H()

    # nonlinear effects
    fd = rbd.ForwardDynamics(p.mb)
    fd.computeC(p.mb, p.mbc);
    h = fd.C()
    
    # compute Minv
    Minv = M.inverse()
    
    # compute lambda1, Jbar1, Nprec1
    lambda1 = inv_spd_sr(J1*Minv*J1.transpose(),1e-3)
    J1bar = Minv*J1.transpose()*lambda1
    Nprec1 = e.MatrixXd.Identity(nDof, nDof) - J1bar*J1
    
    # compute lambda21, Jbar21
    J21 = J2*Nprec1
    lambda21 = inv_spd_sr(J21*Minv*J21.transpose(),1e-3)
    J21bar = Minv*J21.transpose()*lambda21
    Nprec2 = Nprec1 * (e.MatrixXd.Identity(nDof, nDof) - J21bar*J21)
    
#     # change desired position for secondary task
#     if i < MOVEMENT_TIME:
#         xdes = 1 + (i*(10.0-1)/MOVEMENT_TIME)
#         zdes = 1 + (i*(10.0-1)/MOVEMENT_TIME)
    
    # error
    err1 = e.VectorXd(S1.rows())
    err1[0] = 0-T1.coeff(0,3)
    err1[1] = 0-T1.coeff(1,3)
    err1[2] = 0-T1.coeff(2,3)
    derr1 = -J1*dq
    err2 = e.VectorXd(S2.rows())
    err2[0] = T2.coeff(0,3)
    err2[1] = T2.coeff(1,3)
    err2[2] = T2.coeff(2,3)
    derr2 = -J2*dq
    
    # control
    tau_1 = e.VectorXd(J1.transpose()*lambda1*(0*err1+40*derr1-dJ1*dq))
#     tau_2 = e.VectorXd(J21.transpose()*lambda21*(400*err2+40*derr2-dJ2*dq-J2*Minv*tau_1))
#     tau = tau_1 + Nprec1.transpose()*tau_2 + h
    tau = tau_1


    # command system:
    for i in jointsId:
        b.setJointMotorControl2(robotId, i, b.TORQUE_CONTROL, force=tau[i])
    
    b.stepSimulation()
    time.sleep(DT)
b.disconnect()

Imported lbr_iiwa
-0.330072
-0.888205
-0.964839
 0.347583 -0.911883    0.2183  0.330072
 0.894253  0.392395  0.215262  0.888205
-0.281954  0.120394  0.951844  0.964839
        0         0         0         1
-0.330067
-0.888201
-0.964822
  0.34758 -0.911881  0.218313  0.330067
 0.894249  0.392397  0.215273  0.888201
-0.281969  0.120402  0.951839  0.964822
        0         0         0         1
-0.330061
-0.888193
 -0.96479
 0.347576 -0.911877  0.218337  0.330061
 0.894242  0.392403  0.215294  0.888193
-0.281997  0.120415  0.951829   0.96479
        0         0         0         1
-0.330055
-0.888181
-0.964744
 0.347574  -0.91187  0.218369  0.330055
  0.89423  0.392414  0.215323  0.888181
-0.282037  0.120432  0.951815  0.964744
        0         0         0         1
 -0.33005
-0.888163
-0.964687
 0.347574 -0.911861  0.218409   0.33005
 0.894214  0.392431  0.215359  0.888163
-0.282088  0.120451  0.951797  0.964687
        0         0         0         1
-0.330047
-0.888142
 -0.96462
 0

-0.334954
-0.883938
-0.960344
  0.35488 -0.909335  0.217186  0.334954
 0.890524  0.399514  0.217612  0.883938
-0.284651  0.116183  0.951565  0.960344
        0         0         0         1
-0.335177
  -0.8838
-0.960302
 0.355195 -0.909251  0.217024  0.335177
 0.890406  0.399775  0.217618    0.8838
 -0.28463  0.115943    0.9516  0.960302
        0         0         0         1
-0.335405
-0.883661
-0.960263
 0.355515 -0.909166  0.216856  0.335405
 0.890286   0.40004  0.217621  0.883661
-0.284605  0.115696  0.951638  0.960263
        0         0         0         1
-0.335638
 -0.88352
-0.960227
 0.355843 -0.909079  0.216682  0.335638
 0.890164   0.40031  0.217623   0.88352
-0.284576  0.115443  0.951677  0.960227
        0         0         0         1
-0.335875
-0.883377
-0.960192
 0.356177 -0.908991  0.216502  0.335875
 0.890041  0.400584  0.217622  0.883377
-0.284544  0.115183  0.951718  0.960192
        0         0         0         1
-0.336118
-0.883232
-0.960161
 0.356517 -0.908902 

-0.355043
-0.873324
-0.962055
 0.382892   -0.9025  0.197198  0.355043
 0.881263   0.42087  0.215043  0.873324
-0.277071 0.0914453  0.956488  0.962055
        0         0         0         1
-0.355497
-0.873101
-0.962158
 0.383531 -0.902349  0.196649  0.355497
 0.881065   0.42133  0.214955  0.873101
-0.276818 0.0908185  0.956621  0.962158
        0         0         0         1
-0.355954
-0.872876
-0.962264
 0.384174 -0.902196  0.196092  0.355954
 0.880866  0.421792  0.214866  0.872876
-0.276561 0.0901845  0.956755  0.962264
        0         0         0         1
-0.356414
 -0.87265
-0.962373
 0.384822 -0.902043  0.195527  0.356414
 0.880665  0.422256  0.214776   0.87265
-0.276299 0.0895434  0.956891  0.962373
        0         0         0         1
-0.356877
-0.872423
-0.962484
 0.385474 -0.901888  0.194954  0.356877
 0.880463  0.422723  0.214684  0.872423
-0.276033 0.0888949  0.957028  0.962484
        0         0         0         1
-0.357342
-0.872195
-0.962598
  0.38613 -0.901733 

-0.383229
-0.859717
-0.971631
  0.42382 -0.892492  0.154386  0.383229
 0.868916  0.448748   0.20883  0.859717
-0.255659 0.0456421  0.965689  0.971631
        0         0         0         1
-0.383782
-0.859451
 -0.97188
 0.424655 -0.892274  0.153346  0.383782
 0.868669  0.449287  0.208698  0.859451
-0.255112 0.0445825  0.965883   0.97188
        0         0         0         1
-0.384335
-0.859186
-0.972131
 0.425492 -0.892055  0.152296  0.384335
 0.868422  0.449826  0.208565  0.859186
-0.254558 0.0435144  0.966078  0.972131
        0         0         0         1
-0.384888
 -0.85892
-0.972386
 0.426331 -0.891835  0.151236  0.384888
 0.868174  0.450366  0.208432   0.85892
-0.253999 0.0424376  0.966273  0.972386
        0         0         0         1
-0.385443
-0.858653
-0.972643
 0.427173 -0.891613  0.150165  0.385443
 0.867926  0.450906  0.208299  0.858653
-0.253432 0.0413523  0.966469  0.972643
        0         0         0         1
-0.385998
-0.858387
-0.972903
 0.428018 -0.891389 

-0.415931
-0.843746
-0.990858
  0.476202  -0.876191  0.0743058   0.415931
  0.853517   0.480893   0.200626   0.843746
  -0.21152 -0.0321173   0.976846   0.990858
         0          0          0          1
 -0.41647
-0.843475
-0.991259
  0.477123  -0.875833  0.0726013    0.41647
  0.853244   0.481436   0.200486   0.843475
 -0.210545 -0.0337098   0.977003   0.991259
         0          0          0          1
-0.417007
-0.843204
-0.991663
  0.478045  -0.875471  0.0708847   0.417007
   0.85297    0.48198   0.200345   0.843204
 -0.209561 -0.0353115   0.977158   0.991663
         0          0          0          1
-0.417544
-0.842933
 -0.99207
  0.478967  -0.875105   0.069156   0.417544
  0.852695   0.482523   0.200205   0.842933
  -0.20857 -0.0369225    0.97731    0.99207
         0          0          0          1
-0.418079
-0.842662
-0.992478
  0.479889  -0.874735  0.0674153   0.418079
  0.852421   0.483066   0.200065   0.842662
  -0.20757 -0.0385428   0.977461   0.992478
         0    

...

         0          0          0          1
 -1.33647
-0.621356
-0.242326
   0.55274 -0.0302872   0.832803    1.33647
  0.819457  -0.161986  -0.549773   0.621356
  0.151554   0.986328 -0.0647172   0.242326
         0          0          0          1
 -1.33636
-0.621422
  -0.2424
  0.552624 -0.0302062   0.832883    1.33636
   0.81952   -0.16213  -0.549637   0.621422
  0.151638   0.986307 -0.0648421     0.2424
         0          0          0          1
 -1.33625
-0.621489
-0.242475
  0.552507 -0.0301254   0.832964    1.33625
  0.819583  -0.162274    -0.5495   0.621489
  0.151722   0.986286  -0.064967   0.242475
         0          0          0          1
 -1.33613
-0.621556
 -0.24255
   0.55239 -0.0300448   0.833044    1.33613
  0.819646  -0.162418  -0.549364   0.621556
  0.151807   0.986265  -0.065092    0.24255
         0          0          0          1
 -1.33602
-0.621622
-0.242625
  0.552273 -0.0299642   0.833124    1.33602
  0.819709  -0.162562  -0.549227   0.621622
  0.151892   0

-0.246157
  0.547184 -0.0265294   0.836592    1.33104
  0.822373  -0.169075  -0.543246   0.624427
  0.155859   0.985246 -0.0706984   0.246157
         0          0          0          1
 -1.33092
-0.624488
-0.246242
   0.54707 -0.0264534   0.836669    1.33092
  0.822431  -0.169227  -0.543111   0.624488
  0.155954   0.985222 -0.0708226   0.246242
         0          0          0          1
  -1.3308
-0.624557
 -0.24633
  0.546945 -0.0263867   0.836753     1.3308
  0.822496  -0.169373  -0.542967   0.624557
   0.15605   0.985199 -0.0709348    0.24633
         0          0          0          1
 -1.33069
-0.624615
-0.246421
  0.546833 -0.0263134   0.836828    1.33069
  0.822551   -0.16953  -0.542834   0.624615
  0.156151   0.985174 -0.0710605   0.246421
         0          0          0          1
 -1.33058
-0.624674
-0.246507
   0.54672 -0.0262373   0.836904    1.33058
  0.822608  -0.169683  -0.542701   0.624674
  0.156247   0.985149  -0.071186   0.246507
         0          0          0  

 -1.32553
-0.627323
-0.250522
  0.541584 -0.0229058   0.840334    1.32553
  0.825143  -0.176613  -0.536607   0.627323
  0.160705   0.984014 -0.0767502   0.250522
         0          0          0          1
 -1.32542
 -0.62738
-0.250614
   0.54147 -0.0228329    0.84041    1.32542
  0.825197  -0.176768  -0.536472    0.62738
  0.160807   0.983988 -0.0768732   0.250614
         0          0          0          1
  -1.3253
-0.627436
-0.250708
  0.541357 -0.0227608   0.840485     1.3253
  0.825252  -0.176925  -0.536336   0.627436
   0.16091   0.983961 -0.0769962   0.250708
         0          0          0          1
 -1.32519
-0.627493
  -0.2508
  0.541243 -0.0226881    0.84056    1.32519
  0.825307  -0.177081  -0.536201   0.627493
  0.161012   0.983935 -0.0771191     0.2508
         0          0          0          1
 -1.32508
 -0.62755
-0.250894
  0.541129 -0.0226162   0.840635    1.32508
  0.825361  -0.177238  -0.536065    0.62755
  0.161116   0.983908 -0.0772419   0.250894
         0    

-0.255178
  0.536151 -0.0195365   0.843896    1.32016
  0.827673   -0.18423   -0.53011   0.629948
  0.165828   0.982689 -0.0826054   0.255178
         0          0          0          1
 -1.32005
    -0.63
-0.255277
   0.53604 -0.0194666   0.843968    1.32005
  0.827723  -0.184391  -0.529975       0.63
  0.165937    0.98266 -0.0827279   0.255277
         0          0          0          1
 -1.31994
-0.630051
-0.255377
  0.535929 -0.0193971    0.84404    1.31994
  0.827774  -0.184552  -0.529841   0.630051
  0.166047   0.982631 -0.0828502   0.255377
         0          0          0          1
 -1.31983
-0.630103
-0.255478
  0.535817  -0.019328   0.844113    1.31983
  0.827824  -0.184713  -0.529706   0.630103
  0.166157   0.982602 -0.0829724   0.255478
         0          0          0          1
 -1.31972
-0.630155
-0.255579
  0.535705  -0.019259   0.844186    1.31972
  0.827874  -0.184875  -0.529572   0.630155
  0.166268   0.982573 -0.0830943   0.255579
         0          0          0  

 -1.31418
-0.632633
-0.260849
  0.530121 -0.0159466   0.847772    1.31418
  0.830289  -0.193075  -0.522821   0.632633
  0.172021   0.981054 -0.0891129   0.260849
         0          0          0          1
 -1.31407
 -0.63268
-0.260957
  0.530011 -0.0158807   0.847842    1.31407
  0.830336   -0.19324  -0.522686    0.63268
  0.172138   0.981023 -0.0892329   0.260957
         0          0          0          1
 -1.31396
-0.632727
-0.261064
    0.5299  -0.015815   0.847913    1.31396
  0.830382  -0.193405  -0.522552   0.632727
  0.172255   0.980992 -0.0893528   0.261064
         0          0          0          1
 -1.31385
-0.632774
-0.261172
  0.529789 -0.0157494   0.847983    1.31385
  0.830428   -0.19357  -0.522417   0.632774
  0.172372    0.98096 -0.0894726   0.261172
         0          0          0          1
 -1.31374
-0.632821
 -0.26128
  0.529679  -0.015684   0.848053    1.31374
  0.830475  -0.193735  -0.522282   0.632821
   0.17249   0.980928 -0.0895923    0.26128
         0    